In [10]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sksurv.compare import compare_survival
from sksurv.nonparametric import kaplan_meier_estimator

In [11]:
# Lendo os dados
data = pd.read_csv('../data/processed/Processed_Telecom_Data.csv')

Levando em conta os nossos dados, podemos inferir o seguinte:
- Temos dados censurados à esquerda, pois não sabemos quando o nosso 
relacionamento com os clientes começou.
- Também temos dados censorados à direita, representados pelos clientes que
ainda não cancelaram o serviço.

Como primeiro passo, vamos aplicar o teste de Kepler-Meier para estimar a
função de sobrevivência dos nossos dados. Para tal, iremos usar apenas os 
usuários que não deram churn, já que o teste fiunciona bem em dados censurados à
direita. Como esse é um teste não paramétrico, não precisamos nos preocupar com
a distribuição dos dados. Além disso, só podemos usar duas variáveis, uma para
o tempo e outra para o evento, que no nosso caso é o cancelamento do serviço.
Para tal, usaremos a coluna `tenure` e a coluna `Churn`, respectivamente.

### Teste de Kepler-Meier

In [12]:
# Aplicando o teste
tempo, probabilidade = kaplan_meier_estimator(data['Churn'], data['tenure'])

In [13]:
# Plotando o gráfico
fig = go.Figure(data=[go.Scatter(x=tempo, y=probabilidade)])

# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Churn dado o tempo de relacionamento com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier</i></sup>',
    title_font=dict(size=23, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=False
)

A probabilidade geral de sobrevivência é bem alta, se mantendo acima de 50% 
durante todo o período. Nos primeiros 4 meses, a probabilidade do cliente não 
churnar é acima de 90%.

Agora, vamos ver se há diferença entre os grupos de gênero e senioridade.

In [52]:
# Filtrando os dados masculinos
data_male = data.query("gender == 'Male'")

# Aplicando o teste
tempo_male, probabilidade_male = kaplan_meier_estimator(data_male['Churn'], data_male['tenure'])

# Filtrando os dados femininos
data_female = data.query("gender == 'Female'")

# Aplicando o teste
tempo_female, probabilidade_female = kaplan_meier_estimator(data_female['Churn'], data_female['tenure'])

# Plotando o gráfico
fig = go.Figure(data=[go.Scatter(x=tempo_male, y=probabilidade_male, name='Masculino'),
                      go.Scatter(x=tempo_female, y=probabilidade_female, name='Feminino')])


# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Churn dado o tempo de relacionamento com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para os diferentes gêneros</i></sup>',
    title_font=dict(size=23, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)
    

In [51]:
# Comparando os grupos
generos = data['gender']

# Criando um array com o zip de churn e tenure
array_status = np.array(list(zip(data['Churn'].values.astype(bool), data['tenure'].values)), dtype=[('status', bool), ('time', float)])

# Aplicando o teste
chi2, p_valor = compare_survival(array_status, generos)

if p_valor < 0.05:
    print(f"Com um p valor de {p_valor}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {p_valor}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")



Com um p valor de 0.4684173507569699, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


No geral, pessoas do gênero masculino têm uma probabilidade de churn maior do
que pessoas do gênero feminino. No entanto, a diferença não é tão grande. Também 
é possível notar que o cenário se inverte com clientes com um tempo de 
relacionamento superior a 67 meses, com as pessoas do gênero feminino tendo uma
probabilidade de churn maior do que as pessoas do gênero masculino.

Com 95% de confiança, podemos dizer que as curvas de churn para os grupos de
gênero são iguais.

In [25]:
# Filtrando os dados de clientes sênior
data_senior = data.query("SeniorCitizen == 1")

# Aplicando o teste
tempo_senior, probabilidade_senior = kaplan_meier_estimator(data_senior['Churn'], data_senior['tenure'])

# Filtrando os dados de clientes não sênior
data_not_senior = data.query("SeniorCitizen == 0")

# Aplicando o teste
tempo_not_senior, probabilidade_not_senior = kaplan_meier_estimator(data_not_senior['Churn'], data_not_senior['tenure'])

# Plotando o gráfico
fig = go.Figure(data=[go.Scatter(x=tempo_senior, y=probabilidade_senior, name='Senior'),
                      go.Scatter(x=tempo_not_senior, y=probabilidade_not_senior, name='Não Senior')])

# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Churn dado o tempo de relacionamento com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para os diferentes grupos de idade</i></sup>',
    title_font=dict(size=23, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Churn'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)
    

In [53]:
# Comparando os grupos
senioridade = data['SeniorCitizen']

# Aplicando o teste
chi2, p_valor = compare_survival(array_status, senioridade)

if p_valor < 0.05:
    print(f"Com um p valor de {p_valor}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {p_valor}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")



Com um p valor de 1.26761920666699e-25, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


Pessoas mais velhas são menos propensas a dar churn do que pessoas mais novas.
A diferença durante o último mês do período chegou a ser de 20%. Com 95% de
confiança, podemos dizer que as curvas de churn para os grupos de senioridade
são diferentes.

Agora, vamos considerar as variáveis 'Partner' e 'Dependents', para averiguar se
há diferença entre pessoas que moram sozinhas e pessoas que moram com alguém.

In [26]:
# Filtrando os dados de pessoas com parceiros
data_partner = data.query("Partner == 'Yes'")

# Aplicando o teste
tempo_partner, probabilidade_partner = kaplan_meier_estimator(data_partner['Churn'], data_partner['tenure'])

# Filtrando os dados de pessoas sem parceiros
data_not_partner = data.query("Partner == 'No'")

# Aplicando o teste
tempo_not_partner, probabilidade_not_partner = kaplan_meier_estimator(data_not_partner['Churn'], data_not_partner['tenure'])

# Plotando o gráfico
fig = go.Figure(data=[go.Scatter(x=tempo_partner, y=probabilidade_partner, name='Sim'),
                      go.Scatter(x=tempo_not_partner, y=probabilidade_not_partner, name='Não')])


# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Churn dado o tempo de relacionamento com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para pessoas com parceiros</i></sup>',
    title_font=dict(size=23, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Churn'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)
    

In [28]:
# Filtrando os dados de pessoas com dependentes
data_dependents = data.query("Dependents == 'Yes'")

# Aplicando o teste
tempo_dependents, probabilidade_dependents = kaplan_meier_estimator(data_dependents['Churn'], data_dependents['tenure'])

# Filtrando os dados de pessoas sem dependentes
data_not_dependents = data.query("Dependents == 'No'")

# Aplicando o teste
tempo_not_dependents, probabilidade_not_dependents = kaplan_meier_estimator(data_not_dependents['Churn'], data_not_dependents['tenure'])

# Plotando o gráfico
fig = go.Figure(data=[go.Scatter(x=tempo_dependents, y=probabilidade_dependents, name='Sim'),
                      go.Scatter(x=tempo_not_dependents, y=probabilidade_not_dependents, name='Não')])


# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Churn dado o tempo de relacionamento com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para pessoas com dependentes</i></sup>',
    title_font=dict(size=23, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Churn'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)
    

In [22]:
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,False
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,False
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,True
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,False
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,True
